In [1]:
import pandas as pd
import numpy as np

import sklearn
from sklearn import tree

import matplotlib.pyplot as plt

%matplotlib inline

pd.options.display.float_format = '{:,.2f}'.format


from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split

import pickle

In [2]:
df = pd.read_csv('../data/processed/all_rooms_combined.csv', index_col=0)

print(f'Наблюдений в датасете: {df.shape[0]}')

Наблюдений в датасете: 19144


In [3]:
df.head()

,rooms,subway,admin_okrug,district,street,home_number,price,year_of_construction,total_meters,kitchen_meters,flat_type,house_type,"dist_to_subway, min",way_to_subway,is_euro,is_skyscraper,floor_type,wc_count,wc_type,class_real
0,1,Спартак,СЗАО,р-н Покровское-Стрешнево,Алиа ЖК,к7,"16,781,328.00","2,023.00",37.00,11.00,Новостройка,Монолитный,5.00,пешком,False,False,usual,1,совмещенный,премиум
1,1,Шелепиха,СЗАО,р-н Хорошево-Мневники,Шелепихинская набережная,34к7,"16,500,000.00","2,024.00",42.00,12.00,Новостройка,Монолитный,14.00,пешком,False,False,usual,1,совмещенный,бизнес
2,1,Стрешнево,СЗАО,р-н Щукино,Щукинская улица,7/9С7,"20,540,352.00","2,023.00",47.00,16.00,Новостройка,Монолитный,10.00,пешком,False,False,usual,2,совмещенный,премиум
3,1,Шелепиха,СЗАО,р-н Хорошево-Мневники,Шелепихинская набережная,34к3,"20,800,000.00","2,020.00",32.00,11.00,Вторичка,Монолитный,14.00,пешком,False,False,view,1,совмещенный,премиум
4,1,Спартак,СЗАО,р-н Покровское-Стрешнево,Северо-Западный ао,Клубный Город на Реке Примавера ЖК,"30,460,120.00","2,024.00",52.00,11.00,Новостройка,Монолитный,14.00,пешком,False,False,usual,1,раздельный,премиум


In [4]:
df.dtypes

rooms                     int64
subway                   object
admin_okrug              object
district                 object
street                   object
home_number              object
price                   float64
year_of_construction    float64
total_meters            float64
kitchen_meters          float64
flat_type                object
house_type               object
dist_to_subway, min     float64
way_to_subway            object
is_euro                    bool
is_skyscraper              bool
floor_type               object
wc_count                  int64
wc_type                  object
class_real               object
dtype: object

### Разбиваем на X и y, делаем сплит ###

In [5]:
log = ['total_meters', 'kitchen_meters', 'dist_to_subway, min']
categorical = ['admin_okrug', 'district', 'subway', 'is_skyscraper', 'class_real', 'way_to_subway', 'wc_type', 'house_type', 'flat_type', 'floor_type','rooms']
ordinal = ['year_of_construction', 'wc_count']

In [6]:
X = df[log + categorical + ordinal]
y = df['price']/df['total_meters']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, shuffle=True)

In [8]:
X_train.head(2)

,total_meters,kitchen_meters,"dist_to_subway, min",admin_okrug,district,subway,is_skyscraper,class_real,way_to_subway,wc_type,house_type,flat_type,floor_type,rooms,year_of_construction,wc_count
7694,52.00,11.00,7.00,САО,р-н Аэропорт,Динамо,False,премиум,пешком,совмещенный,Монолитный,Вторичка Апартаменты,usual,2,"2,020.00",1
9773,101.00,17.00,12.00,ЮЗАО,р-н Обручевский,Калужская,False,премиум,пешком,совмещенный,Монолитный,Новостройка,usual,3,"2,023.00",1


In [9]:
X_train.isna().sum()

total_meters               0
kitchen_meters             0
dist_to_subway, min        2
admin_okrug                0
district                   0
subway                     0
is_skyscraper              0
class_real                 0
way_to_subway              2
wc_type                    0
house_type              1706
flat_type                  0
floor_type                 0
rooms                      0
year_of_construction       0
wc_count                   0
dtype: int64

In [322]:
X_train.house_type = X_train.house_type.fillna('No')
X_train.year_of_construction = X_train.year_of_construction.fillna(-1)
X_train['dist_to_subway, min'] = X_train['dist_to_subway, min'].fillna(20)
X_train.way_to_subway = X_train.way_to_subway.fillna('пешком')

In [323]:
X_test.house_type = X_test.house_type.fillna('No')
X_test.year_of_construction = X_test.year_of_construction.fillna(-1)
X_test['dist_to_subway, min'] = X_test['dist_to_subway, min'].fillna(20)
X_test.way_to_subway = X_test.way_to_subway.fillna('пешком')

In [324]:
X_test.isna().sum()

total_meters            0
kitchen_meters          0
dist_to_subway, min     0
admin_okrug             0
district                0
subway                  0
is_skyscraper           0
class_real              0
way_to_subway           0
wc_type                 0
house_type              0
flat_type               0
floor_type              0
rooms                   0
year_of_construction    0
wc_count                0
dtype: int64

In [325]:
def log_transform(x):
    return np.log(x + 1)

In [326]:
log_transformer = FunctionTransformer(log_transform)

In [327]:
col_transformer = ColumnTransformer([("Log transform", log_transformer, log),
                                ("Scale", StandardScaler(), ordinal),
                                ("One hot", OneHotEncoder(sparse=False, handle_unknown='ignore'),categorical)],
                                remainder="passthrough")
X_train_transformed = col_transformer.fit_transform(X_train)

In [328]:
X_test_transformed = col_transformer.transform(X_test)

In [329]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import catboost as ctb

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [330]:
df_test = pd.merge(X_test, pd.DataFrame(y_test, columns=['real']), how = 'left', left_index=True, right_index=True)

### Простая линейная регрессия ###

In [331]:
linreg = LinearRegression().fit(X_train_transformed, y_train)
MAE_lr = round(mean_absolute_error(y_test, linreg.predict(X_test_transformed)), 3)
MAE_lr

3522816778525.98

In [332]:
df_test['preds'] = linreg.predict(X_test_transformed)
df_test.head()

,total_meters,kitchen_meters,"dist_to_subway, min",admin_okrug,district,subway,is_skyscraper,class_real,way_to_subway,wc_type,house_type,flat_type,floor_type,rooms,year_of_construction,wc_count,real,preds
16769,7.00,2.00,13.00,ЗАО,р-н Солнцево,Солнцево,False,бизнес,пешком,совмещенный,Блочный,Вторичка Апартаменты,ground,9,"1,978.00",1,"321,428.57","251,776.00"
2532,37.00,10.00,7.00,ЗАО,р-н Раменки,Раменки,False,бизнес,пешком,совмещенный,Блочный,Вторичка,usual,1,"1,980.00",1,"327,027.03","374,016.00"
11934,169.00,25.00,9.00,ЦАО,р-н Пресненский,Выставочная,True,элитный,пешком,совмещенный,Монолитный,Новостройка,sky,5,"2,023.00",1,"1,202,130.18","1,020,416.00"
7643,42.00,10.00,1.00,ЦАО,р-н Басманный,Электрозаводская,False,премиум,пешком,совмещенный,Монолитный,Новостройка,view,2,"2,024.00",1,"442,837.14","467,712.00"
17832,21.00,5.00,15.00,ЮАО,р-н Даниловский,ЗИЛ,False,премиум,пешком,совмещенный,Монолитный,Новостройка,usual,9,"2,023.00",1,"677,945.52","493,824.00"


In [333]:
df_test['diff'] = df_test['real']-df_test['preds']

In [334]:
df_test.sort_values(by='diff')

,total_meters,kitchen_meters,"dist_to_subway, min",admin_okrug,district,subway,is_skyscraper,class_real,way_to_subway,wc_type,house_type,flat_type,floor_type,rooms,year_of_construction,wc_count,real,preds,diff
3048,39.00,9.00,6.00,ЮЗАО,р-н Северное Бутово,Битца,False,комфорт,на транспорте,раздельный,Монолитный,Вторичка,usual,1,"1,991.00",1,"266,666.67","9,211,118,237,937,472.00","-9,211,118,237,670,806.00"
12215,210.00,20.00,7.00,ЦАО,р-н Якиманка,Третьяковская,False,элитный,пешком,совмещенный,Монолитный,Вторичка,usual,5,"2,000.00",3,"952,380.95","1,842,112.00","-889,731.05"
11676,186.00,17.00,5.00,ЦАО,р-н Якиманка,Третьяковская,False,премиум,пешком,совмещенный,Кирпичный,Вторичка,ground,5,"1,912.00",2,"586,005.38","1,392,192.00","-806,186.62"
14907,103.00,13.00,10.00,ЦАО,р-н Хамовники,Киевская,False,элитный,пешком,раздельный,Кирпичный,Вторичка,usual,5,"1,963.00",1,"718,446.60","1,461,632.00","-743,185.40"
13581,190.00,20.00,6.00,ЦАО,р-н Басманный,Чистые пруды,False,премиум,пешком,совмещенный,No,Вторичка,ground,5,"1,917.00",2,"421,052.63","1,090,048.00","-668,995.37"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1910,43.00,10.00,16.00,ЮАО,р-н Бирюлево Восточное,Бирюлёво-Пассажирская,False,комфорт,пешком,раздельный,Монолитный,Вторичка,view,1,"2,013.00",1,"259,302.33","-222,738,685,036,352.00","222,738,685,295,654.31"
9662,50.00,7.00,5.00,СВАО,р-н Лосиноостровский,Лось,False,эконом,на транспорте,раздельный,Панельный,Вторичка,usual,3,"1,969.00",1,"230,000.00","-222,738,685,132,480.00","222,738,685,362,480.00"
16477,17.00,2.00,6.00,ЗАО,р-н Можайский,Немчиновка,False,эконом,на транспорте,совмещенный,Панельный,Вторичка Апартаменты,ground,9,"2,007.00",1,"235,294.12","-222,738,685,127,296.00","222,738,685,362,590.12"
17197,18.00,3.00,16.00,СВАО,р-н Ярославский,Лось,False,комфорт,на транспорте,совмещенный,Монолитный,Вторичка Апартаменты,usual,9,"2,019.00",1,"272,222.22","-222,738,685,101,824.00","222,738,685,374,046.22"


### Ridge-регрессия ###

In [335]:
ridge = Ridge(alpha=1000).fit(X_train_transformed, y_train)
MAE_r = round(mean_absolute_error(y_test, ridge.predict(X_test_transformed)), 3)
MAE_r

72832.266

In [336]:
df_test['preds'] = ridge.predict(X_test_transformed)
df_test.head()

,total_meters,kitchen_meters,"dist_to_subway, min",admin_okrug,district,subway,is_skyscraper,class_real,way_to_subway,wc_type,house_type,flat_type,floor_type,rooms,year_of_construction,wc_count,real,preds,diff
16769,7.00,2.00,13.00,ЗАО,р-н Солнцево,Солнцево,False,бизнес,пешком,совмещенный,Блочный,Вторичка Апартаменты,ground,9,"1,978.00",1,"321,428.57","266,751.58","69,652.57"
2532,37.00,10.00,7.00,ЗАО,р-н Раменки,Раменки,False,бизнес,пешком,совмещенный,Блочный,Вторичка,usual,1,"1,980.00",1,"327,027.03","394,136.28","-46,988.97"
11934,169.00,25.00,9.00,ЦАО,р-н Пресненский,Выставочная,True,элитный,пешком,совмещенный,Монолитный,Новостройка,sky,5,"2,023.00",1,"1,202,130.18","900,738.29","181,714.18"
7643,42.00,10.00,1.00,ЦАО,р-н Басманный,Электрозаводская,False,премиум,пешком,совмещенный,Монолитный,Новостройка,view,2,"2,024.00",1,"442,837.14","598,745.25","-24,874.86"
17832,21.00,5.00,15.00,ЮАО,р-н Даниловский,ЗИЛ,False,премиум,пешком,совмещенный,Монолитный,Новостройка,usual,9,"2,023.00",1,"677,945.52","433,710.34","184,121.52"


In [337]:
df_test['diff'] = df_test['real']-df_test['preds']
df_test.sort_values(by='diff')

,total_meters,kitchen_meters,"dist_to_subway, min",admin_okrug,district,subway,is_skyscraper,class_real,way_to_subway,wc_type,house_type,flat_type,floor_type,rooms,year_of_construction,wc_count,real,preds,diff
14116,388.00,30.00,9.00,ЦАО,р-н Якиманка,Шаболовская,False,комфорт,пешком,совмещенный,Монолитный,Вторичка Пентхаус,view,5,"2,006.00",3,"283,247.42","642,784.21","-359,536.79"
12084,225.00,33.00,7.00,ЦАО,р-н Якиманка,Октябрьская,False,бизнес,пешком,совмещенный,Кирпичный,Вторичка,usual,5,"1,900.00",2,"311,111.11","605,938.45","-294,827.34"
11945,352.00,32.00,16.00,ЦАО,р-н Пресненский,Улица 1905 года,False,элитный,пешком,совмещенный,No,Вторичка Пентхаус,usual,5,"2,016.00",5,"707,386.36","978,014.94","-270,628.58"
14910,183.00,18.00,1.00,ЦАО,р-н Басманный,Бауманская,False,комфорт,пешком,совмещенный,No,Вторичка,usual,5,"1,902.00",2,"284,153.01","551,459.30","-267,306.29"
13842,337.00,18.00,7.00,ЦАО,р-н Хамовники,Парк Культуры,False,элитный,пешком,совмещенный,Кирпичный,Вторичка,ground,5,"2,010.00",3,"714,674.05","979,881.53","-265,207.48"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4571,234.00,60.00,7.00,ЦАО,р-н Хамовники,Киевская,False,элитный,на транспорте,раздельный,Монолитный,Новостройка,ground,2,"2,026.00",4,"2,390,384.62","1,008,214.02","1,382,170.60"
10461,82.00,6.00,13.00,ЦАО,р-н Хамовники,Киевская,False,элитный,пешком,совмещенный,Монолитный,Новостройка,usual,3,"2,024.00",2,"2,344,408.59","959,647.33","1,384,761.25"
10444,120.00,12.00,1.00,ЦАО,р-н Тверской,Лубянка,False,элитный,пешком,совмещенный,No,Вторичка Апартаменты,ground,3,-1.00,3,"2,337,411.54","912,599.90","1,424,811.64"
13558,302.00,10.00,6.00,ЦАО,р-н Пресненский,Пушкинская,False,элитный,пешком,совмещенный,No,Вторичка,usual,5,"2,006.00",2,"2,423,663.58","918,247.38","1,505,416.19"


### Lasso регрессия ###

In [338]:
lasso_reg = Lasso(alpha = 2.65e-05).fit(X_train_transformed, y_train)
MAE_lasso = round(mean_absolute_error(y_test, lasso_reg.predict(X_test_transformed)), 3)
MAE_lasso

/Users/max/miniforge3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.611e+13, tolerance: 9.618e+10
  model = cd_fast.enet_coordinate_descent(


51557.355

In [339]:
df_test['preds'] = lasso_reg.predict(X_test_transformed)
df_test.head()

,total_meters,kitchen_meters,"dist_to_subway, min",admin_okrug,district,subway,is_skyscraper,class_real,way_to_subway,wc_type,house_type,flat_type,floor_type,rooms,year_of_construction,wc_count,real,preds,diff
16769,7.00,2.00,13.00,ЗАО,р-н Солнцево,Солнцево,False,бизнес,пешком,совмещенный,Блочный,Вторичка Апартаменты,ground,9,"1,978.00",1,"321,428.57","252,000.82","54,676.99"
2532,37.00,10.00,7.00,ЗАО,р-н Раменки,Раменки,False,бизнес,пешком,совмещенный,Блочный,Вторичка,usual,1,"1,980.00",1,"327,027.03","374,224.16","-67,109.26"
11934,169.00,25.00,9.00,ЦАО,р-н Пресненский,Выставочная,True,элитный,пешком,совмещенный,Монолитный,Новостройка,sky,5,"2,023.00",1,"1,202,130.18","1,020,441.05","301,391.89"
7643,42.00,10.00,1.00,ЦАО,р-н Басманный,Электрозаводская,False,премиум,пешком,совмещенный,Монолитный,Новостройка,view,2,"2,024.00",1,"442,837.14","467,605.26","-155,908.11"
17832,21.00,5.00,15.00,ЮАО,р-н Даниловский,ЗИЛ,False,премиум,пешком,совмещенный,Монолитный,Новостройка,usual,9,"2,023.00",1,"677,945.52","494,032.64","244,235.18"


In [340]:
df_test['diff'] = df_test['real']-df_test['preds']
df_test.sort_values(by='diff')

,total_meters,kitchen_meters,"dist_to_subway, min",admin_okrug,district,subway,is_skyscraper,class_real,way_to_subway,wc_type,house_type,flat_type,floor_type,rooms,year_of_construction,wc_count,real,preds,diff
12215,210.00,20.00,7.00,ЦАО,р-н Якиманка,Третьяковская,False,элитный,пешком,совмещенный,Монолитный,Вторичка,usual,5,"2,000.00",3,"952,380.95","1,842,204.76","-889,823.80"
11676,186.00,17.00,5.00,ЦАО,р-н Якиманка,Третьяковская,False,премиум,пешком,совмещенный,Кирпичный,Вторичка,ground,5,"1,912.00",2,"586,005.38","1,392,215.23","-806,209.86"
14907,103.00,13.00,10.00,ЦАО,р-н Хамовники,Киевская,False,элитный,пешком,раздельный,Кирпичный,Вторичка,usual,5,"1,963.00",1,"718,446.60","1,461,601.30","-743,154.69"
13581,190.00,20.00,6.00,ЦАО,р-н Басманный,Чистые пруды,False,премиум,пешком,совмещенный,No,Вторичка,ground,5,"1,917.00",2,"421,052.63","1,090,061.73","-669,009.10"
9060,150.00,15.00,10.00,ЦАО,р-н Якиманка,Третьяковская,False,элитный,пешком,совмещенный,Монолитный,Вторичка Апартаменты,ground,3,"2,019.00",2,"1,193,333.33","1,827,172.04","-633,838.70"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12405,207.00,18.00,5.00,ЗАО,р-н Раменки,Ломоносовский проспект,False,элитный,на транспорте,совмещенный,Монолитный,Новостройка,usual,5,"2,021.00",1,"2,006,280.19","1,136,378.39","869,901.80"
9604,202.00,12.00,6.00,ЦАО,р-н Хамовники,Парк Культуры,False,элитный,пешком,совмещенный,Кирпичный,Вторичка,ground,3,"2,009.00",1,"2,289,287.87","1,188,211.99","1,101,075.88"
10444,120.00,12.00,1.00,ЦАО,р-н Тверской,Лубянка,False,элитный,пешком,совмещенный,No,Вторичка Апартаменты,ground,3,-1.00,3,"2,337,411.54","1,064,237.55","1,273,173.99"
13558,302.00,10.00,6.00,ЦАО,р-н Пресненский,Пушкинская,False,элитный,пешком,совмещенный,No,Вторичка,usual,5,"2,006.00",2,"2,423,663.58","1,050,510.35","1,373,153.23"


### Decision Tree ###

In [341]:
dt_reg = DecisionTreeRegressor(random_state = 0, max_depth=15).fit(X_train_transformed, y_train)
MAE_dt = round(mean_absolute_error(y_test, dt_reg.predict(X_test_transformed)), 3)
MAE_dt

36255.778

In [342]:
df_test['preds'] = dt_reg.predict(X_test_transformed)
df_test.head()

,total_meters,kitchen_meters,"dist_to_subway, min",admin_okrug,district,subway,is_skyscraper,class_real,way_to_subway,wc_type,house_type,flat_type,floor_type,rooms,year_of_construction,wc_count,real,preds,diff
16769,7.00,2.00,13.00,ЗАО,р-н Солнцево,Солнцево,False,бизнес,пешком,совмещенный,Блочный,Вторичка Апартаменты,ground,9,"1,978.00",1,"321,428.57","292,307.69","69,427.75"
2532,37.00,10.00,7.00,ЗАО,р-н Раменки,Раменки,False,бизнес,пешком,совмещенный,Блочный,Вторичка,usual,1,"1,980.00",1,"327,027.03","322,765.30","-47,197.14"
11934,169.00,25.00,9.00,ЦАО,р-н Пресненский,Выставочная,True,элитный,пешком,совмещенный,Монолитный,Новостройка,sky,5,"2,023.00",1,"1,202,130.18","1,167,453.48","181,689.13"
7643,42.00,10.00,1.00,ЦАО,р-н Басманный,Электрозаводская,False,премиум,пешком,совмещенный,Монолитный,Новостройка,view,2,"2,024.00",1,"442,837.14","466,528.78","-24,768.12"
17832,21.00,5.00,15.00,ЮАО,р-н Даниловский,ЗИЛ,False,премиум,пешком,совмещенный,Монолитный,Новостройка,usual,9,"2,023.00",1,"677,945.52","571,209.45","183,912.89"


In [343]:
df_test['diff'] = df_test['real']-df_test['preds']
df_test.sort_values(by='diff')

,total_meters,kitchen_meters,"dist_to_subway, min",admin_okrug,district,subway,is_skyscraper,class_real,way_to_subway,wc_type,house_type,flat_type,floor_type,rooms,year_of_construction,wc_count,real,preds,diff
15170,125.00,7.00,8.00,ЦАО,р-н Пресненский,Тверская,False,элитный,пешком,совмещенный,No,Вторичка,usual,5,"1,913.00",2,"768,000.00","2,240,000.00","-1,472,000.00"
6257,204.00,10.00,6.00,ЦАО,р-н Арбат,Смоленская,False,элитный,пешком,совмещенный,Кирпичный,Новостройка,usual,2,"2,019.00",1,"852,500.00","1,951,950.00","-1,099,450.00"
8767,142.00,44.00,5.00,ЦАО,р-н Якиманка,Третьяковская,False,элитный,пешком,совмещенный,Монолитный,Новостройка,ground,3,"2,024.00",3,"1,601,126.76","2,607,172.41","-1,006,045.65"
11673,239.00,82.00,12.00,ЦАО,р-н Хамовники,Спортивная,False,элитный,пешком,совмещенный,Монолитный,Новостройка Апартаменты,usual,5,"2,024.00",4,"1,614,965.25","2,462,185.13","-847,219.88"
14299,164.00,18.00,6.00,ЗАО,р-н Раменки,Киевская,False,элитный,на транспорте,совмещенный,Монолитный,Вторичка,sky,5,"2,011.00",1,"701,219.51","1,500,000.00","-798,780.49"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13558,302.00,10.00,6.00,ЦАО,р-н Пресненский,Пушкинская,False,элитный,пешком,совмещенный,No,Вторичка,usual,5,"2,006.00",2,"2,423,663.58","1,540,573.91","883,089.66"
10459,110.00,4.00,13.00,ЦАО,р-н Хамовники,Киевская,False,элитный,пешком,совмещенный,Монолитный,Новостройка,usual,3,"2,024.00",2,"2,329,340.05","1,391,251.86","938,088.18"
10461,82.00,6.00,13.00,ЦАО,р-н Хамовники,Киевская,False,элитный,пешком,совмещенный,Монолитный,Новостройка,usual,3,"2,024.00",2,"2,344,408.59","1,391,251.86","953,156.72"
9604,202.00,12.00,6.00,ЦАО,р-н Хамовники,Парк Культуры,False,элитный,пешком,совмещенный,Кирпичный,Вторичка,ground,3,"2,009.00",1,"2,289,287.87","1,013,344.86","1,275,943.01"


**Среди базовых моделей лучший результат без настройки гиперпараметров по метрике MAE показывают решающие деревья. Проведем настройку гиперпараметров по кросс-валидации**

In [308]:
from sklearn.model_selection import GridSearchCV

In [309]:
criterion = ['squared_error', 'absolute_error']
splitter = ['best', 'random']
max_depth = [5, 10, 20, 30, 40, 50]

In [310]:
grid = [{'criterion':criterion,
        'splitter':splitter,
        'max_depth':max_depth}]

gs = GridSearchCV(estimator=DecisionTreeRegressor(),
                  param_grid = grid,
                  scoring='neg_mean_absolute_error',
                  cv = 10,
                  n_jobs = -1)

In [311]:
gs = gs.fit(X_train_transformed, y_train)

KeyboardInterrupt: 

In [ ]:
-1*gs.best_score_

In [ ]:
gs.best_params_

In [ ]:
gs.score(X_test_transformed, y_test)

In [ ]:
gs.best_estimator_

In [ ]:
df_test['preds'] = gs.best_estimator_.predict(X_test_transformed)
df_test.head()

In [ ]:
df_test['diff'] = df_test['real']-df_test['preds']
df_test.sort_values(by='diff')

In [ ]:
df_test['price_real'] = df_test['total_meters'] * df_test['real']
df_test['price_predicted'] = df_test['total_meters'] * df_test['preds']
df_test['diff'] = df_test['price_real'] - df_test['price_predicted']

In [ ]:
df_test

### CatBoost ###

In [344]:
X_train_ctb = X_train.copy()
X_test_ctb = X_test.copy()

In [345]:
X_train_ctb.dtypes

total_meters            float64
kitchen_meters          float64
dist_to_subway, min     float64
admin_okrug              object
district                 object
subway                   object
is_skyscraper              bool
class_real               object
way_to_subway            object
wc_type                  object
house_type               object
flat_type                object
floor_type               object
rooms                     int64
year_of_construction    float64
wc_count                  int64
dtype: object

In [346]:
X_test_ctb.dtypes

total_meters            float64
kitchen_meters          float64
dist_to_subway, min     float64
admin_okrug              object
district                 object
subway                   object
is_skyscraper              bool
class_real               object
way_to_subway            object
wc_type                  object
house_type               object
flat_type                object
floor_type               object
rooms                     int64
year_of_construction    float64
wc_count                  int64
dtype: object

In [347]:
X_train_ctb.is_skyscraper = X_train_ctb.is_skyscraper.astype(str)
X_test_ctb.is_skyscraper = X_test_ctb.is_skyscraper.astype(str)

In [348]:
X_train_ctb.dtypes == 'O'

total_meters            False
kitchen_meters          False
dist_to_subway, min     False
admin_okrug              True
district                 True
subway                   True
is_skyscraper            True
class_real               True
way_to_subway            True
wc_type                  True
house_type               True
flat_type                True
floor_type               True
rooms                   False
year_of_construction    False
wc_count                False
dtype: bool

In [349]:
ctbst = ctb.CatBoostRegressor(cat_features = list(X_train_ctb.columns[X_train_ctb.dtypes == 'O']),
                                 random_state = 42)
ctbst.fit(X_train_ctb, y_train, verbose = False)

In [350]:
ctbst_preds = ctbst.predict(X_test_ctb)

In [351]:
df_test['preds'] = ctbst_preds
df_test.head()

,total_meters,kitchen_meters,"dist_to_subway, min",admin_okrug,district,subway,is_skyscraper,class_real,way_to_subway,wc_type,house_type,flat_type,floor_type,rooms,year_of_construction,wc_count,real,preds,diff
16769,7.00,2.00,13.00,ЗАО,р-н Солнцево,Солнцево,False,бизнес,пешком,совмещенный,Блочный,Вторичка Апартаменты,ground,9,"1,978.00",1,"321,428.57","330,245.34","29,120.88"
2532,37.00,10.00,7.00,ЗАО,р-н Раменки,Раменки,False,бизнес,пешком,совмещенный,Блочный,Вторичка,usual,1,"1,980.00",1,"327,027.03","340,301.43","4,261.73"
11934,169.00,25.00,9.00,ЦАО,р-н Пресненский,Выставочная,True,элитный,пешком,совмещенный,Монолитный,Новостройка,sky,5,"2,023.00",1,"1,202,130.18","1,102,017.99","34,676.70"
7643,42.00,10.00,1.00,ЦАО,р-н Басманный,Электрозаводская,False,премиум,пешком,совмещенный,Монолитный,Новостройка,view,2,"2,024.00",1,"442,837.14","446,144.33","-23,691.64"
17832,21.00,5.00,15.00,ЮАО,р-н Даниловский,ЗИЛ,False,премиум,пешком,совмещенный,Монолитный,Новостройка,usual,9,"2,023.00",1,"677,945.52","521,543.14","106,736.07"


In [352]:
MAE_ctbst = round(mean_absolute_error(y_test, ctbst_preds), 3)
MAE_ctbst

37014.392

In [353]:
ctbst2 = ctb.CatBoostRegressor(iterations = 1000, 
                             depth = 15, 
                             learning_rate = 0.04746000096201897,
                             random_strength = 1,
                             border_count = 254,
                             l2_leaf_reg = 3, 
                             grow_policy = 'SymmetricTree',
                             cat_features = list(X_train_ctb.columns[X_train_ctb.dtypes == 'O']),
                             random_state = 42)
ctbst2.fit(X_train_ctb, y_train, verbose = False)

In [354]:
ctbst2_preds = ctbst2.predict(X_test_ctb)
MAE_ctbst2 = round(mean_absolute_error(y_test, ctbst2_preds), 3)
MAE_ctbst2

30640.308

In [355]:
df_test['preds'] = ctbst2_preds
df_test.head()

,total_meters,kitchen_meters,"dist_to_subway, min",admin_okrug,district,subway,is_skyscraper,class_real,way_to_subway,wc_type,house_type,flat_type,floor_type,rooms,year_of_construction,wc_count,real,preds,diff
16769,7.00,2.00,13.00,ЗАО,р-н Солнцево,Солнцево,False,бизнес,пешком,совмещенный,Блочный,Вторичка Апартаменты,ground,9,"1,978.00",1,"321,428.57","333,359.82","29,120.88"
2532,37.00,10.00,7.00,ЗАО,р-н Раменки,Раменки,False,бизнес,пешком,совмещенный,Блочный,Вторичка,usual,1,"1,980.00",1,"327,027.03","355,830.93","4,261.73"
11934,169.00,25.00,9.00,ЦАО,р-н Пресненский,Выставочная,True,элитный,пешком,совмещенный,Монолитный,Новостройка,sky,5,"2,023.00",1,"1,202,130.18","1,142,332.71","34,676.70"
7643,42.00,10.00,1.00,ЦАО,р-н Басманный,Электрозаводская,False,премиум,пешком,совмещенный,Монолитный,Новостройка,view,2,"2,024.00",1,"442,837.14","448,525.82","-23,691.64"
17832,21.00,5.00,15.00,ЮАО,р-н Даниловский,ЗИЛ,False,премиум,пешком,совмещенный,Монолитный,Новостройка,usual,9,"2,023.00",1,"677,945.52","581,938.66","106,736.07"


In [393]:
import pickle
with open("ctbst_reg.pkl", "wb") as f:
    pickle.dump(ctbst2, f)

### RandomForest Regressor ###

In [356]:
rf = RandomForestRegressor(criterion='squared_error', n_estimators = 300)

In [357]:
rf.fit(X_train_transformed, y_train)

RandomForestRegressor(n_estimators=300)

In [358]:
rf_preds = rf.predict(X_test_transformed)
MAE_rf = round(mean_absolute_error(y_test, rf_preds), 3)
MAE_rf

28520.647

In [362]:
X_full = X.copy()

In [363]:
X_full.house_type = X_full.house_type.fillna('No')
X_full.year_of_construction = X_full.year_of_construction.fillna(-1)
X_full['dist_to_subway, min'] = X_full['dist_to_subway, min'].fillna(20)
X_full.way_to_subway = X_full.way_to_subway.fillna('пешком')

In [364]:
X_transformed = col_transformer.fit_transform(X_full)

In [365]:
rf.fit(X_transformed, y)

RandomForestRegressor(n_estimators=300)

In [367]:
rf_full_preds = rf.predict(X_transformed)
MAE_rf_full = round(mean_absolute_error(y, rf_full_preds), 3)
MAE_rf_full

11173.097

In [397]:

with open("rf_reg.pkl", "wb") as f:
    pickle.dump(rf, f)

In [369]:
with open("col_transf.pkl", "wb") as f:
    pickle.dump(col_transformer, f)

In [370]:
with open("rf_reg.pkl", "rb") as f:
    model = pickle.load(f)

In [371]:
with open("col_transf.pkl", "rb") as f:
    column_transformer = pickle.load(f)

In [384]:
sample_test = {
    'total_meters':[58.2, 60.0],
    'kitchen_meters':[21.0, 12.0],
    'dist_to_subway, min':[15.0, 25.0],
    'admin_okrug':['СВАО', 'СВАО'],
    'subway':['Ботанический сад', 'ВДНХ'],
    'is_skyscraper': ['False', 'False'],
    'class_real':['комфорт', ''],
    'way_to_subway':['пешком', 'пешком'],
    'wc_type':['совмещенный', 'раздельный'],
    'house_type':['Монолитный', 'кирпичный'],
    'flat_type':['Новостройка', 'Вторичка'],
    'rooms': [2, 2],
    'year_of_construction':[2021, 1995],
    'wc_count':[2, 1],
    'district':['р-н Останкинский', 'р-н Алексеевский'],
    'floor_type':['usual', 'usual']
}

In [385]:
sample_df = pd.DataFrame.from_dict(sample_test)
sample_df

,total_meters,kitchen_meters,"dist_to_subway, min",admin_okrug,subway,is_skyscraper,class_real,way_to_subway,wc_type,house_type,flat_type,rooms,year_of_construction,wc_count,district,floor_type
0,58.20,21.00,15.00,СВАО,Ботанический сад,False,комфорт,пешком,совмещенный,Монолитный,Новостройка,2,2021,2,р-н Останкинский,usual
1,60.00,12.00,25.00,СВАО,ВДНХ,False,,пешком,раздельный,кирпичный,Вторичка,2,1995,1,р-н Алексеевский,usual


In [386]:
sample_df_transformed = column_transformer.transform(sample_df)

In [387]:
model.predict(sample_df_transformed)[0] * 60

15167801.457003517

### Ansamble ###

In [394]:
ansamble_preds = (0.8 * rf_preds + 0.2*ctbst2_preds)

In [395]:
MAE_ans = round(mean_absolute_error(y_test, ansamble_preds), 3)
MAE_ans

28331.901